In [202]:
import pandas as pd
import numpy as np
import re
import wikipedia
import datetime
import requests
from dateutil.relativedelta import relativedelta
import matplotlib.pyplot as plt
import json
import urllib
from urllib.parse import urlencode
from urllib.request import urlopen
from collections import deque
# documentation: https://mwparserfromhell.readthedocs.io/en/latest/api/mwparserfromhell.nodes.html#module-mwparserfromhell.nodes.wikilink
import mwparserfromhell
import networkx as nx
import random

In [203]:
%matplotlib inline

In [3]:
def getUserActivity(article, granularity, start, end, project ="en.wikipedia.org",
                    access="all-access", agent="user",dateformat="iso"):
    """
    Method to obtain user activity of a given page for a given period of time
    article: name of the wikiipedia article
    granularity: time granularity of activity, either 'monthly' or 'daily'
    start: start date of the research as Datetime.datetime object
    end: end date of the research as Datetime.datetime object
    project: If you want to filter by project, use the domain of any Wikimedia project (by default en.wikipedia.org)
    access: If you want to filter by access method, use one of desktop, mobile-app or mobile-web (by default all-access)
    agent: If you want to filter by agent type, use one of user, bot or spider (by default user).
    dateformat: the dateformat used in result array, can be 'iso','ordinal','datetime'.
    return:
        it return an array of array of the form [ [user_activity_value1, date1], [user_activity_value2, date2]]
    """

    #granularity['monthly','daily']
    #format['iso','ordinal','datetime']
    #Be carefull, for daily granularity left bound date is included, for monthly granularity left bound date is excluded
    
    dstart = start.strftime("%Y%m%d")
    dend = end.strftime("%Y%m%d")
    path = ("https://wikimedia.org/api/rest_v1/metrics/pageviews/per-article/"+project
            +"/"+access+"/"+agent+"/"+article+"/"+granularity+"/"+dstart+"/"+dend)
    r = requests.get(path)
    res = []
    for i in range(len(r.json()['items'])):
        time_label = None
        if granularity == 'daily':
            time_label = (start + datetime.timedelta(days=i))
        else:
            time_label = (start + relativedelta(months=+i))
        if dateformat == 'iso':
            time_label = time_label.isoformat()
        elif dateformat == 'ordinal':
            time_label = time_label.toordinal()
            
        res.append([r.json()['items'][i]['views'],time_label])
    return res

In [140]:
API_URL = "https://en.wikipedia.org/w/api.php"
# TODO: make it deal with server side error 
def parse_with_date(title, date=None):
    '''
    
    :param title: title of the wikipedia article
    :param date: snapshot of the page as it was on the date. If None, scrapes the page as is now. 
    :return: the parsed page of wikipedia
    '''
    data = {"action": "query", "prop": "revisions", "rvlimit": 1,
            "rvprop": "content", "format": "json", "titles": title}
    if date is not None:
        data["rvstart"] = date
    raw = urlopen(API_URL, urlencode(data).encode()).read()
    res = json.loads(raw)
    try:
        text = list(res["query"]["pages"].values())[0]["revisions"][0]["*"]
    except KeyError as err:
        print("Key error".format(err))
        print(title)
        return None
    return mwparserfromhell.parse(text)

In [5]:
seed = 'Stan Lee'

In [6]:
stan_lee_12_nov = parse_with_date(seed, date= '2018-11-13T00:00:00Z')
stan_lee_12_nov_l = stan_lee_12_nov.filter_wikilinks()

In [7]:
# 1st hop links. 
stan_lee_12_nov_l

['[[New York City]]',
 '[[Los Angeles]]',
 '[[California]]',
 '[[Jack Kirby]]',
 '[[Steve Ditko]]',
 '[[John Romita Sr.]]',
 '[[Don Heck]]',
 '[[Bill Everett]]',
 '[[Joe Maneely]]',
 '[[The Will Eisner Award Hall of Fame]]',
 '[[Jack Kirby Hall of Fame]]',
 '[[National Medal of Arts]]',
 '[[Joan Boocock Lee]]',
 '[[#LeeMair2002|Lee & Mair 2002, p. 27]]',
 '[[Marvel Comics]]',
 '[[U.S. Securities and Exchange Commission|Securities and Exchange Commission]]',
 '[[multimedia]]',
 '[[Jack Kirby]]',
 '[[Steve Ditko]]',
 '[[character (arts)|characters]]',
 '[[Spider-Man]]',
 '[[Hulk (comics)|Hulk]]',
 '[[Doctor Strange]]',
 '[[Fantastic Four]]',
 '[[Daredevil (Marvel Comics character)|Daredevil]]',
 '[[Black Panther (Marvel Comics)|Black Panther]]',
 '[[X-Men]]',
 '[[Larry Lieber]]',
 '[[Ant-Man]]',
 '[[Iron Man]]',
 '[[Thor (Marvel Comics)|Thor]]',
 '[[Comics Code Authority]]',
 '[[Marvel Cinematic Universe]]',
 '[[Will Eisner Award Hall of Fame]]',
 '[[Jack Kirby Hall of Fame]]',
 '[[Natio

In [235]:
def preprocess_links(links):
    non_link = re.compile('Category:|File:|wikt:|.*#.*')
    links = [str(link.title) for link in links if non_link.match(str(link.title))==None]
    links = list(set(links))
    return links

In [94]:
links = preprocess_links(stan_lee_12_nov_l)
links

['Robert Greenberger',
 'Michael Chabon',
 'naturalism (literature)',
 'Mark Ruffalo',
 'Robbie Robertson (comics)',
 'Marvel Comics#Editors-in-chief',
 'Comic-Con International',
 'The Hollywood Reporter',
 'KTTV',
 'The Daily Telegraph',
 'Newsday',
 'Inkpot Award',
 'Mystic Comics',
 'Horror comics',
 'Disney Legends',
 'Gary Friedrich',
 'Fantastic Four',
 'Hiroyuki Takei',
 'Marvel Legends',
 'cover-date',
 'Chris Bachalo',
 'Washington Heights, Manhattan',
 'IGN',
 'Marvel Universe',
 'Amazing Heroes',
 'Tales to Astonish',
 'elder abuse',
 'SEC Rule 10b-5',
 'Merry Marvel Marching Society',
 'Silver Surfer',
 'Jean Giraud',
 'Captain America (1990 film)',
 'Flash (comics)',
 'City News Service',
 'Alex Ross',
 'Stripperella',
 'List of pseudonyms',
 'TCL Chinese Theatre',
 'press releases',
 'List of American comics creators',
 'National Public Radio',
 'Marvel Cinematic Universe',
 'Jack Kirby Hall of Fame',
 'The Comics Journal',
 'Producers Guild of America',
 'Jessica Jones'

In [95]:
len(links), type(stan_lee_12_nov_l[0].title)

(370, mwparserfromhell.wikicode.Wikicode)

### Sampling
A heuristic we could use for sampling is to give more probability to the links at the front of the page (assuming they might be more important and more related to the subject) than to the other links.

In [134]:
def sample_links(n_links, n_links_sample, percentage_l, percentage_l_subsample):
    '''
    Example : make the first 20% of the links to account for 50% of the subsample
    :param n_links: number of links a page has
    :param n_links_sample: number of links you want to sample
    :param percentage_l: consider the top percentage_l links ..
    :param percentage_l_subsample: to account for percentage_l_subsample of the subsampled links
    :return:  the chosen links to select
    NOTE: if percentage_l = percentage_l_subsample => aprox uniform sampling 
    '''
    # no sampling needed here
    if n_links_sample >= n_links:
        return np.array(range(n_links))
    
    # how many links from the first group should be subsampled
    n_links_first_group = int(n_links_sample * percentage_l_subsample)
    # how many links from the first group we have
    n_links_first_group_pop = int(n_links * percentage_l)
    
    # no sampling from the first group
    if n_links_first_group_pop < n_links_first_group:
        links_chosen_group_1 = np.array(range(n_links_first_group_pop))
        remaining = n_links_sample - n_links_first_group_pop
        links_chosen_group_2 = np.random.choice(range(n_links_first_group_pop+1, n_links), size = remaining, replace= False)
        return np.append(links_chosen_group_1, links_chosen_group_2)
    
    # if we have to sample from both groups, create the probabilities 
    perc_1 = percentage_l_subsample / n_links_first_group_pop
    perc_2 = (1 - percentage_l_subsample) / (n_links - n_links_first_group_pop)
    #print('Links from first group were sampled with p ', perc_1, ' and links from second group were sampled with p ', perc_2)
    p = [perc_1] * n_links_first_group_pop + [perc_2] * (n_links - n_links_first_group_pop)
    chosen_links = np.random.choice(n_links, size = n_links_sample, p = p, replace=False)
    return chosen_links
    
    

### test

In [83]:
sample_links(100, 10, 0.1, 1)

Links from first group were sampled with p  0.1  and links from second group were sampled with p  0.0


array([4, 5, 0, 7, 2, 1, 3, 9, 8, 6])

In [84]:
sample_links(100, 10, 0.09, 1)

array([ 0,  1,  2,  3,  4,  5,  6,  7,  8, 58])

In [87]:
# first 10 links should account for 50% of the subsampled links
sample_links(100, 10, 0.1, 0.5)

Links from first group were sampled with p  0.05  and links from second group were sampled with p  0.005555555555555556


array([61,  2,  0,  3,  9,  8,  1, 37, 67,  7])

### Crawling algorithm (BFS-like)

In [179]:
seed = 'Stan Lee'
date = '2018-11-13T00:00:00Z'

In [184]:
# CONSTANTS: to be explored
max_nodes = 500
n_links_hop1 = 175 # how many direct neighbours
n_links_indirect_min = 10
n_links_indirect_max = 25
min_popularity = 30 # the minimum number of links a page should have to be taken into the graph (to avoid stubs or really small articles)



In [185]:
# init algo
queue = deque([seed])
G = nx.DiGraph()
while(G.number_of_nodes() < max_nodes):
    if len(queue) == 0:
        print('no more links to dequeue')
        break
    article = queue.popleft()
    # crawl it
    article_content = parse_with_date(title=article, date=date)
    # check parse succesfull
    if article_content is not None:
        links = article_content.filter_wikilinks()
        links = preprocess_links(links)
        print(len(links))
        if (len(links) > min_popularity):
            G.add_node(article)
            print('Added ', article, ' to the graph')
            if article == seed:
                idx_chosen_links = sample_links(n_links=len(links), n_links_sample=n_links_hop1, percentage_l=0.1, percentage_l_subsample=0.5)
            else:
                '''
                if len(links) > 100:
                    n_links_s = 50 # this is a hub, so get more links
                else:
                    n_links_s = random.randint(n_links_indirect_min, n_links_indirect_max)
                '''
                n_links_s = random.randint(n_links_indirect_min, n_links_indirect_max)
                idx_chosen_links = sample_links(n_links=len(links), n_links_sample=n_links_s, percentage_l=0.4, percentage_l_subsample=0.5 )
            for idx in idx_chosen_links:
                if links[idx] not in G.nodes() and links[idx] not in queue:
                    queue.append(links[idx])
        if (G.number_of_nodes() % 10 == 0):
            print('Number of nodes is ', G.number_of_nodes())
            
            
            

369
Added  Stan Lee  to the graph
69
Added  Dan Goodwin  to the graph
103
Added  What If (comics)  to the graph
60
Added  Comics Buyer's Guide  to the graph
79
Added  Robert Bernstein (comics)  to the graph
1
439
Added  Horror comics  to the graph
251
Added  Disney Legends  to the graph
1
152
Added  French New Wave  to the graph
459
Added  Marvel Cinematic Universe  to the graph
1
1
1
134
Added  Larry Lieber  to the graph
Number of nodes is  10
23
Number of nodes is  10
195
Added  Atlas Comics (1950s)  to the graph
161
Added  Google Books  to the graph
1
425
Added  Marvel Comics#Editors-in-chief  to the graph
8
153
Added  Don Heck  to the graph
236
Added  Joe Kubert  to the graph
127
Added  Carnegie Hall  to the graph
238
Added  Hank Pym  to the graph
674
Added  Secret Files and Origins  to the graph
543
Added  Marvel Legends  to the graph
114
Added  action figure  to the graph
Number of nodes is  20
8
Number of nodes is  20
186
Added  Archie Goodwin (comics)  to the graph
68
Added  Ep

146
Added  Time (magazine)  to the graph
44
Added  Josei manga  to the graph
360
Added  1993 in comics  to the graph
45
Added  Ghosts (comics)  to the graph
378
Added  Neil Gaiman  to the graph
1
1
250
Added  Blade (comics)  to the graph
Key error
Masters of Terror
90
Added  Richard Corben  to the graph
1
33
Added  Kathryn Beaumont  to the graph
23
45
Added  John Lounsbery  to the graph
Key error
#Class of 1996
367
Added  Oprah Winfrey  to the graph
Number of nodes is  190
14
Number of nodes is  190
275
Added  Peter Jennings  to the graph
149
Added  Richard Fleischer  to the graph
34
Added  Adriana Caselotti  to the graph
92
Added  Glen Keane  to the graph
51
Added  New German Cinema  to the graph
47
Added  Philippe Garrel  to the graph
433
Added  Alfred Hitchcock  to the graph
62
Added  Andrew Sarris  to the graph
1
89
Added  Stéphane Audran  to the graph
109
Added  Roger Vadim  to the graph
Number of nodes is  200
299
Added  iconoclasm  to the graph
645
Added  Film noir  to the graph

165
Added  marketing  to the graph
50
Added  Toy Biz  to the graph
Number of nodes is  350
1
Number of nodes is  350
35
Added  Phosphorescence  to the graph
80
Added  Captain Power and the Soldiers of the Future  to the graph
1
341
Added  film  to the graph
38
Added  1:6 scale modeling  to the graph
54
Added  ToyFare  to the graph
67
Added  thermochromism  to the graph
1
479
Added  Jean Giraud  to the graph
45
Added  Scott Hampton  to the graph
163
Added  Cerebus the Aardvark  to the graph
269
Added  comic strip  to the graph
Number of nodes is  360
48
Added  Our Fighting Forces  to the graph
276
Added  Gerry Conway  to the graph
218
Added  Bob Clampett  to the graph
14
187
Added  Akira (manga)  to the graph
599
Added  Tulsa, Oklahoma  to the graph
31
Added  Iron Man and Sub-Mariner  to the graph
58
Added  Controller (Marvel Comics)  to the graph
207
Added  The Amazing Spider-Man  to the graph
35
Added  work for hire  to the graph
130
Added  Barry Windsor-Smith  to the graph
Number of 

In [186]:
G.number_of_nodes()

500

In [187]:
# now a second pass is needed to add the edges (since this is a directed graph, cannot do in 1 pass)
for article in G.nodes():
    article_content = parse_with_date(title=article, date=date)
    links = article_content.filter_wikilinks()
    links = preprocess_links(links)
    for link in links:
        if link in G.nodes():
            G.add_edge(article, link)
            print('add edge from ', article, ' to ', link)

add edge from  Stan Lee  to  Robert Greenberger
add edge from  Stan Lee  to  Mark Ruffalo
add edge from  Stan Lee  to  Robbie Robertson (comics)
add edge from  Stan Lee  to  Marvel Comics#Editors-in-chief
add edge from  Stan Lee  to  The Hollywood Reporter
add edge from  Stan Lee  to  KTTV
add edge from  Stan Lee  to  The Daily Telegraph
add edge from  Stan Lee  to  Newsday
add edge from  Stan Lee  to  Mystic Comics
add edge from  Stan Lee  to  Horror comics
add edge from  Stan Lee  to  Disney Legends
add edge from  Stan Lee  to  Gary Friedrich
add edge from  Stan Lee  to  Fantastic Four
add edge from  Stan Lee  to  Hiroyuki Takei
add edge from  Stan Lee  to  Marvel Legends
add edge from  Stan Lee  to  Chris Bachalo
add edge from  Stan Lee  to  Washington Heights, Manhattan
add edge from  Stan Lee  to  IGN
add edge from  Stan Lee  to  Marvel Universe
add edge from  Stan Lee  to  Amazing Heroes
add edge from  Stan Lee  to  Tales to Astonish
add edge from  Stan Lee  to  elder abuse
add e

add edge from  Comics Buyer's Guide  to  Jack Kirby
add edge from  Comics Buyer's Guide  to  Alex Toth
add edge from  Comics Buyer's Guide  to  Tony Isabella
add edge from  Comics Buyer's Guide  to  United States
add edge from  Comics Buyer's Guide  to  Rocket's Blast Comicollector
add edge from  Comics Buyer's Guide  to  Maggie Thompson
add edge from  Comics Buyer's Guide  to  Peter David
add edge from  Comics Buyer's Guide  to  tabloid (newspaper format)
add edge from  Comics Buyer's Guide  to  comic book
add edge from  Comics Buyer's Guide  to  Alter Ego (magazine)
add edge from  Comics Buyer's Guide  to  John Jackson Miller
add edge from  Robert Bernstein (comics)  to  Tales of Suspense
add edge from  Robert Bernstein (comics)  to  East Farmingdale, New York
add edge from  Robert Bernstein (comics)  to  Larry Lieber
add edge from  Robert Bernstein (comics)  to  George Papp
add edge from  Robert Bernstein (comics)  to  Delray Beach, Florida
add edge from  Robert Bernstein (comics)  

add edge from  Larry Lieber  to  United States
add edge from  Larry Lieber  to  The New York Times
add edge from  Larry Lieber  to  Stan Lee
add edge from  Larry Lieber  to  The Amazing Spider-Man
add edge from  Larry Lieber  to  Iron Man
add edge from  Larry Lieber  to  science fiction
add edge from  Larry Lieber  to  Dick Ayers
add edge from  Larry Lieber  to  New York (state)
add edge from  Larry Lieber  to  Gary Friedrich
add edge from  Larry Lieber  to  Fantastic Four
add edge from  Larry Lieber  to  Tales of Suspense
add edge from  Larry Lieber  to  Don Rico
add edge from  Larry Lieber  to  Art Students League of New York
add edge from  Larry Lieber  to  Washington Heights, Manhattan
add edge from  Larry Lieber  to  Marvel Universe
add edge from  Larry Lieber  to  Tales to Astonish
add edge from  Larry Lieber  to  Balder (comics)
add edge from  Larry Lieber  to  DC Comics
add edge from  Larry Lieber  to  Brooklyn
add edge from  Larry Lieber  to  Jack Kirby
add edge from  Larry Li

add edge from  Don Heck  to  annual publication
add edge from  Don Heck  to  Abrams Books
add edge from  Don Heck  to  Pete Morisi
add edge from  Don Heck  to  Stan Lee
add edge from  Don Heck  to  The Amazing Spider-Man
add edge from  Don Heck  to  Iron Man
add edge from  Don Heck  to  inker
add edge from  Don Heck  to  Fantastic Four
add edge from  Don Heck  to  Mandarin (comics)
add edge from  Don Heck  to  John Romita Sr.
add edge from  Don Heck  to  Tales of Suspense
add edge from  Don Heck  to  Harvey Comics
add edge from  Don Heck  to  Havok (comics)
add edge from  Don Heck  to  Tales to Astonish
add edge from  Don Heck  to  Wonder Woman
add edge from  Don Heck  to  Gerry Conway
add edge from  Don Heck  to  DC Comics
add edge from  Don Heck  to  Brooklyn
add edge from  Don Heck  to  Jack Kirby
add edge from  Don Heck  to  Detective Comics
add edge from  Don Heck  to  Tony Isabella
add edge from  Don Heck  to  Strange Tales
add edge from  Don Heck  to  The Avengers (comic book)
a

add edge from  action figure  to  Captain Power and the Soldiers of the Future
add edge from  action figure  to  1:6 scale modeling
add edge from  action figure  to  Toy Biz
add edge from  action figure  to  Marvel Legends
add edge from  action figure  to  film
add edge from  action figure  to  Ball-jointed doll
add edge from  action figure  to  marketing
add edge from  action figure  to  National Entertainment Collectibles Association
add edge from  action figure  to  thermochromism
add edge from  action figure  to  DC Comics
add edge from  action figure  to  ToyFare
add edge from  action figure  to  comic book
add edge from  action figure  to  superhero
add edge from  action figure  to  Phosphorescence
add edge from  action figure  to  Mattel
add edge from  action figure  to  Marvel Comics
add edge from  Archie Goodwin (comics)  to  Abrams Books
add edge from  Archie Goodwin (comics)  to  Heavy Metal (magazine)
add edge from  Archie Goodwin (comics)  to  Haunted Tank
add edge from  A

...

add edge from  work for hire  to  United States
add edge from  work for hire  to  copyright
add edge from  work for hire  to  Microsoft Windows
add edge from  work for hire  to  Microsoft
add edge from  Barry Windsor-Smith  to  Abrams Books
add edge from  Barry Windsor-Smith  to  Superman
add edge from  Barry Windsor-Smith  to  Bernie Wrightson
add edge from  Barry Windsor-Smith  to  Stan Lee
add edge from  Barry Windsor-Smith  to  Iron Man
add edge from  Barry Windsor-Smith  to  Fantastic Four
add edge from  Barry Windsor-Smith  to  Dave Sim
add edge from  Barry Windsor-Smith  to  work for hire
add edge from  Barry Windsor-Smith  to  DC Comics
add edge from  Barry Windsor-Smith  to  Jack Kirby
add edge from  Barry Windsor-Smith  to  The Comics Journal
add edge from  Barry Windsor-Smith  to  List of comics solicited but never published
add edge from  Barry Windsor-Smith  to  Chamber of Darkness
add edge from  Barry Windsor-Smith  to  graphic novel
add edge from  Barry Windsor-Smith  to

add edge from  Wolfgang Amadeus Phoenix  to  The Washington Post
add edge from  Wolfgang Amadeus Phoenix  to  Time (magazine)
add edge from  Wolfgang Amadeus Phoenix  to  The A.V. Club
add edge from  The People v. O. J. Simpson: American Crime Story  to  Time (magazine)
add edge from  The People v. O. J. Simpson: American Crime Story  to  The Hollywood Reporter
add edge from  The People v. O. J. Simpson: American Crime Story  to  Twitter
add edge from  The People v. O. J. Simpson: American Crime Story  to  Ryan Murphy (writer)
add edge from  The People v. O. J. Simpson: American Crime Story  to  condominium
add edge from  My Beautiful Dark Twisted Fantasy  to  Time (magazine)
add edge from  My Beautiful Dark Twisted Fantasy  to  The New York Times
add edge from  My Beautiful Dark Twisted Fantasy  to  The Hollywood Reporter
add edge from  My Beautiful Dark Twisted Fantasy  to  New York (magazine)
add edge from  My Beautiful Dark Twisted Fantasy  to  IGN
add edge from  My Beautiful Dark 

add edge from  George Tenet  to  President of the United States
add edge from  George Tenet  to  The New York Times
add edge from  George Tenet  to  The Daily Telegraph
add edge from  George Tenet  to  Cold War
add edge from  George Tenet  to  New York (state)
add edge from  George Tenet  to  The Washington Post
add edge from  George Tenet  to  George W. Bush
add edge from  George Tenet  to  Osama bin Laden
add edge from  George Tenet  to  Doubleday (publisher)
add edge from  George Tenet  to  New York City
add edge from  George Tenet  to  Queens
add edge from  Social Security debate in the United States  to  The New York Times
add edge from  Social Security debate in the United States  to  George W. Bush
add edge from  Social Security debate in the United States  to  Memorial Day
add edge from  Osama bin Laden  to  World Trade Center (1973–2001)
add edge from  Osama bin Laden  to  New Jersey
add edge from  Osama bin Laden  to  Al-Qaeda
add edge from  Osama bin Laden  to  1993 World Tr

add edge from  Victor Herbert  to  New York City
add edge from  William A. Seiter  to  film producer
add edge from  William A. Seiter  to  New York City
add edge from  1946 in film  to  Alfred Hitchcock
add edge from  1952 in film  to  High Noon
add edge from  1952 in film  to  Jerry Lewis
add edge from  1952 in film  to  Errol Flynn
add edge from  1952 in film  to  Brigitte Bardot
add edge from  1952 in film  to  Richard Fleischer
add edge from  1952 in film  to  Marie Wilson (American actress)
add edge from  Clem McCarthy  to  New York City
add edge from  Clem McCarthy  to  New York (state)
add edge from  China Clipper (1936 film)  to  The New York Times
add edge from  China Clipper (1936 film)  to  Marie Wilson (American actress)
add edge from  SF Sketchfest  to  Adam McKay
add edge from  SF Sketchfest  to  Wil Wheaton
add edge from  SF Sketchfest  to  The Onion
add edge from  SF Sketchfest  to  United States
add edge from  MTV  to  World Trade Center (1973–2001)
add edge from  MTV 

add edge from  Matt Serra  to  Huntington, New York
add edge from  Irish Americans in New York City  to  Newsday
add edge from  Irish Americans in New York City  to  New York (state)
add edge from  Irish Americans in New York City  to  Brooklyn
add edge from  Irish Americans in New York City  to  New York City
add edge from  Irish Americans in New York City  to  Queens
add edge from  Huntington, New York  to  The New York Times
add edge from  Huntington, New York  to  Newsday
add edge from  Huntington, New York  to  New York (magazine)
add edge from  Huntington, New York  to  comic strip
add edge from  Huntington, New York  to  Long Island
add edge from  Putnam County, New York  to  New York (state)
add edge from  Putnam County, New York  to  New Jersey
add edge from  Putnam County, New York  to  New York City
add edge from  Putnam County, New York  to  Woodrow Wilson
add edge from  Putnam County, New York  to  Long Island
add edge from  New York Lizards  to  Long Island
add edge from 

In [188]:
G.number_of_edges()

5675

In [193]:
# save the graph (take around half h to make)
# nx.write_gpickle(G, "graph.gpickle")

In [245]:
# test it was saved
G2 = nx.read_gpickle("graph.gpickle")

In [246]:
G2.number_of_edges(), G2.number_of_nodes()

(5675, 500)

In [247]:
#G2.nodes() == G.nodes()

## Adding the delta page_view signal

In [252]:
# page views are crawled at the end of the day ! 
views={}

In [249]:
# example
start = datetime.datetime(year=2018,month=11,day=11, hour=23, minute = 59, second = 59)
end =  datetime.datetime(year=2018,month=11,day=12, hour=23, minute = 59, second = 59)
daily_view = getUserActivity(article="Stan Lee",granularity="daily",start=start,end=end,dateformat="iso")
daily_view

[[6929, '2018-11-11T23:59:59'], [2404717, '2018-11-12T23:59:59']]

In [250]:
daily_view[1][0]

2404717

In [253]:
nodes_not_taken = []
for node in G2.nodes():
    try:
        views[node] = getUserActivity(article=node, granularity="daily",start=start,end=end,dateformat="iso")
    except KeyError:
        nodes_not_taken.append(node)
        print(node)

Marvel Comics#Editors-in-chief
action figure
executive producer
condominium
elder abuse
news agency
graphic novel
funny animal
superhero
Not Brand Echh#Legacy
What The--?!
comic book
tabloid (newspaper format)
Aquagirl#Lisa Morel
impresario
science fiction
comics anthology
horror film
iconoclasm
Cinema of Portugal#1960s.E2.80.931970s
Tomorrowland#Disneyland (California)
Atlas Comics (1950s)#Superhero
Atlas Comics (1950s)#Misc.
Atlas Comics (1950s)#Hero
web crawler
horror fiction
spy fiction
Marvel Super-Heroes (comics)#Other iterations
annual publication
inker
USS Stevens (DD-479)#In comic books
Harvey Award#The Jack Kirby Hall of Fame
planetarium
artificial intelligence
court-martial
Sonar (comics)#Successor
Fury (DC Comics)#Helena Kosmatos
Superman Red/Superman Blue
comic strip
work for hire
popular culture
Structure of the United States Army#Branches and Functional Areas
unlawful combatant
Unite the Right rally#President Trump's statements
disarmament
cause célèbre
executive order
f

In [259]:
views_graph = {}
for node in G2.nodes():
    if node in views.keys():
        try:
            views_graph[node] = views[node][1][0] -  views[node][0][0] 
        except IndexError:
            views_graph[node] = views[node][0][0]
    else:
        views_graph[node] = 0

In [260]:
views_graph

{'Stan Lee': 2397788,
 'Dan Goodwin': 3,
 'What If (comics)': 271,
 "Comics Buyer's Guide": 39,
 'Robert Bernstein (comics)': 28,
 'Horror comics': 58,
 'Disney Legends': 1186,
 'French New Wave': 472,
 'Marvel Cinematic Universe': 10095,
 'Larry Lieber': 19144,
 'Atlas Comics (1950s)': 962,
 'Google Books': 150,
 'Marvel Comics#Editors-in-chief': 0,
 'Don Heck': 5783,
 'Joe Kubert': 258,
 'Carnegie Hall': -63,
 'Hank Pym': 379,
 'Secret Files and Origins': 70,
 'Marvel Legends': 42,
 'action figure': 0,
 'Archie Goodwin (comics)': 132,
 'Epic Illustrated': 116,
 'The A.V. Club': -16,
 'United States Army': 541,
 'George W. Bush': 15650,
 'executive producer': 0,
 'Marie Wilson (American actress)': 113,
 'Bonnie Burton': 39,
 'Kyle Baker': -20,
 'Long Island': 226,
 'Marvel Universe': 2236,
 'Silver Surfer': 2283,
 'Kevin Maguire (artist)': 54,
 'Joe Simon': 2794,
 'KTTV': 31,
 'Justice League': 834,
 'Amazing Heroes': -6,
 'The Galactus Trilogy': 619,
 'Peter David': 237,
 'Washington

In [268]:
isinstance(views_graph,dict)

True

In [270]:
nx.set_node_attributes(G2, values= views_graph, name = 'delta')

In [271]:
# check
G2.node['Stan Lee']

{'delta': 2397788}

In [272]:
# save again 
nx.write_gpickle(G2, "graph_with_delta_signal.gpickle")